In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
import optuna
from sklearn.metrics import roc_auc_score
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
import pickle

In [2]:
train = pd.read_csv("/kaggle/input/playground-series-s4e3/train.csv")
train_original = pd.read_csv("/kaggle/input/steel-faults/faults.csv")

train.drop(columns=['id'], axis=1, inplace=True)
target_cols = ['Pastry','Z_Scratch','K_Scatch','Stains','Dirtiness','Bumps','Other_Faults']
train = train[train[target_cols].sum(axis=1) == 1]

In [3]:
test = pd.concat([train_original, train], axis=0)

In [4]:
def feature_generator(data):

    "A function to generate additional features"

    epsilon = 1e-6  # A small constant to avoid division by zero or taking the logarithm of zero
    # Location Features
    data['X_Distance'] = data['X_Maximum'] - data['X_Minimum']
    data['Y_Distance'] = data['Y_Maximum'] - data['Y_Minimum']

    # Density Feature
    data['Density'] = data['Pixels_Areas'] / (data['X_Perimeter'] + data['Y_Perimeter'])

    # Relative Perimeter Feature
    data['Relative_Perimeter'] = data['X_Perimeter'] / (data['X_Perimeter'] + data['Y_Perimeter'] + epsilon)

    # Circularity Feature
    data['Circularity'] = data['Pixels_Areas'] / (data['X_Perimeter'] ** 2)

    # Symmetry Index Feature
    data['Symmetry_Index'] = np.abs(data['X_Distance'] - data['Y_Distance']) / (data['X_Distance'] + data['Y_Distance'] + epsilon)
    
    # Color Contrast Feature
    data['Color_Contrast'] = data['Maximum_of_Luminosity'] - data['Minimum_of_Luminosity']

    # Combined Geometric Index Feature
    data['Combined_Geometric_Index'] = data['Edges_Index'] * data['Square_Index']

    # Interaction Term Feature
    data['X_Distance*Pixels_Areas'] = data['X_Distance'] * data['Pixels_Areas']
    
    # Additional Features
    data['sin_orientation'] = np.sin(data['Orientation_Index'])
    data['Edges_Index2'] = np.exp(data['Edges_Index'] + epsilon)
    data['X_Maximum2'] = np.sin(data['X_Maximum'])
    data['Y_Minimum2'] = np.sin(data['Y_Minimum'])
    data['Aspect_Ratio_Pixels'] = np.where(data['Y_Perimeter'] == 0, 0, data['X_Perimeter'] / data['Y_Perimeter'])
    data['Aspect_Ratio'] = np.where(data['Y_Distance'] == 0, 0, data['X_Distance'] / data['Y_Distance'])

    # Average Luminosity Feature
    data['Average_Luminosity'] = (data['Sum_of_Luminosity'] + data['Minimum_of_Luminosity']) / 2
    
    # Normalized Steel Thickness Feature
    data['Normalized_Steel_Thickness'] = (data['Steel_Plate_Thickness'] - data['Steel_Plate_Thickness'].min()) / (data['Steel_Plate_Thickness'].max() - data['Steel_Plate_Thickness'].min())

    # Logarithmic Features
    data['Log_Perimeter'] = np.log(data['X_Perimeter'] + data['Y_Perimeter'] + epsilon)
    data['Log_Luminosity'] = np.log(data['Sum_of_Luminosity'] + epsilon)
    data['Log_Aspect_Ratio'] = np.log(data['Aspect_Ratio'] ** 2 + epsilon)

    # Statistical Features
    data['Combined_Index'] = data['Orientation_Index'] * data['Luminosity_Index']
    data['Sigmoid_Areas'] = 1 / (1 + np.exp(-data['LogOfAreas'] + epsilon))

    return data

test = feature_generator(test)

In [5]:
# Initialize the scaler
scaler = RobustScaler()

# Identify discrete features
discrete_features = ['TypeOfSteel_A300', 'TypeOfSteel_A400']

# Identify continuous features
continuous_features = [col for col in test.columns if col not in discrete_features + target_cols]

# Fit and transform the scaler on training data
test[continuous_features] = scaler.fit_transform(test[continuous_features])

In [6]:
X = test.drop(columns=target_cols, axis=1)
y1 = test['Pastry']
y2 = test['Z_Scratch']
y3 = test['K_Scatch']
y4 = test['Stains']
y5 = test['Dirtiness']
y6 = test['Bumps']
y7 = test['Other_Faults']

In [7]:
X_train_Pastry, X_test_pastry, y_train_Pastry, y_test_pastry = train_test_split(X, y1, test_size=0.3, random_state=24)
X_train_Z_Scratch, X_test_Z_Scratch, y_train_Z_Scratch, y_test_Z_Scratch = train_test_split(X, y2, test_size=0.3, random_state=24)
X_train_K_Scatch, X_test_K_Scatch, y_train_K_Scatch, y_test_K_Scatch = train_test_split(X, y3, test_size=0.3, random_state=24)
X_train_Stains, X_test_Stains, y_train_Stains, y_test_Stains = train_test_split(X, y4, test_size=0.3, random_state=24)
X_train_Dirtiness, X_test_Dirtiness, y_train_Dirtiness, y_test_Dirtiness = train_test_split(X, y5, test_size=0.3, random_state=24)
X_train_Bumps, X_test_Bumps, y_train_Bumps, y_test_Bumps = train_test_split(X, y6, test_size=0.3, random_state=24)
X_train_Other_Faults, X_test_Other_Faults, y_train_Other_Faults, y_test_Other_Faults = train_test_split(X, y7, test_size=0.3, random_state=24)

In [8]:
# Assuming target_splits is the dictionary containing splits for each target
target_splits = {
    'Pastry': (X_train_Pastry, X_test_pastry, y_train_Pastry, y_test_pastry),
    'Z_Scratch': (X_train_Z_Scratch, X_test_Z_Scratch, y_train_Z_Scratch, y_test_Z_Scratch),
    'K_Scatch': (X_train_K_Scatch, X_test_K_Scatch, y_train_K_Scatch, y_test_K_Scatch),
    'Stains': (X_train_Stains, X_test_Stains, y_train_Stains, y_test_Stains),
    'Dirtiness': (X_train_Dirtiness, X_test_Dirtiness, y_train_Dirtiness, y_test_Dirtiness),
    'Bumps': (X_train_Bumps, X_test_Bumps, y_train_Bumps, y_test_Bumps),
    'Other_Faults': (X_train_Other_Faults, X_test_Other_Faults, y_train_Other_Faults, y_test_Other_Faults)
}

In [9]:
def objective_lgbm(trial, X_train, X_test, y_train, y_test):
    """
    Objective function to be minimized.
    """
    lgbm_params = {
        "objective": "binary",
        "metric": "auc",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "num_class": 1,  # Changed to 1 for binary classification
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
    }
    
    model_lgbm = LGBMClassifier(**lgbm_params)
    model_lgbm.fit(X_train, y_train)
    predictions_lgbm = model_lgbm.predict(X_test)
    accuracy_lgbm = roc_auc_score(y_test, predictions_lgbm)
    return accuracy_lgbm


# Dictionary to store best models for each target
best_models_lgbm = {}

# Loop over each target
for target, (X_train, X_test, y_train, y_test) in target_splits.items():
    print(f"Tuning LGBM for target: {target}")
    study_lgbm = optuna.create_study(direction='maximize')
    study_lgbm.optimize(lambda trial: objective_lgbm(trial, X_train, X_test, y_train, y_test), n_trials=30)
    print('Best hyperparameters:', study_lgbm.best_params)
    print('Best accuracy:', study_lgbm.best_value)

    # Define the best hyperparameters obtained from Optuna
    best_params_lgbm = study_lgbm.best_params

    # Instantiate LGBMClassifier with the best hyperparameters
    best_lgbm_model = LGBMClassifier(**best_params_lgbm, silent=True)

    # Fit the model on the training data
    best_lgbm_model.fit(X_train, y_train)

    # Save the model to a file
    model_filename = f"best_lightgbm_model_{target}.pkl"
    with open(model_filename, 'wb') as f:
        pickle.dump(best_lgbm_model, f)

    # Store the model in the dictionary
    best_models_lgbm[target] = best_lgbm_model

[I 2024-03-22 07:18:13,036] A new study created in memory with name: no-name-5dafadc7-ae90-49c3-a6de-b97bd7c4c130


Tuning LGBM for target: Pastry


[I 2024-03-22 07:18:14,522] Trial 0 finished with value: 0.5798268725562071 and parameters: {'lambda_l1': 1.5689065443135128e-07, 'lambda_l2': 0.0007743185215599293, 'num_leaves': 230, 'feature_fraction': 0.6953609519810082, 'bagging_fraction': 0.6406438423216039, 'bagging_freq': 6, 'min_child_samples': 86}. Best is trial 0 with value: 0.5798268725562071.
[I 2024-03-22 07:18:16,275] Trial 1 finished with value: 0.5742729716520039 and parameters: {'lambda_l1': 0.0002997160889889964, 'lambda_l2': 2.505082865149474e-07, 'num_leaves': 210, 'feature_fraction': 0.8882400143855423, 'bagging_fraction': 0.4669629003943588, 'bagging_freq': 7, 'min_child_samples': 67}. Best is trial 0 with value: 0.5798268725562071.
[I 2024-03-22 07:18:17,684] Trial 2 finished with value: 0.5681665444770284 and parameters: {'lambda_l1': 0.011382947436812198, 'lambda_l2': 0.24888502781122243, 'num_leaves': 229, 'feature_fraction': 0.409666348352522, 'bagging_fraction': 0.7735128553224304, 'bagging_freq': 5, 'min_c

Best hyperparameters: {'lambda_l1': 0.002565363377063284, 'lambda_l2': 0.0023972954455110497, 'num_leaves': 172, 'feature_fraction': 0.939940765287339, 'bagging_fraction': 0.5035878689258695, 'bagging_freq': 1, 'min_child_samples': 64}
Best accuracy: 0.5902145191837732
[LightGBM] [Warning] Unknown parameter: silent
[LightGBM] [Warning] feature_fraction is set=0.939940765287339, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.939940765287339
[LightGBM] [Warning] lambda_l2 is set=0.0023972954455110497, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.0023972954455110497
[LightGBM] [Warning] lambda_l1 is set=0.002565363377063284, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.002565363377063284
[LightGBM] [Warning] bagging_fraction is set=0.5035878689258695, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5035878689258695
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1

[I 2024-03-22 07:19:14,265] A new study created in memory with name: no-name-0d8eac54-21b0-4e99-9a83-6efdd6c1defb


...

Best hyperparameters: {'lambda_l1': 0.003109085161653011, 'lambda_l2': 0.002326714730678544, 'num_leaves': 32, 'feature_fraction': 0.9693790705619205, 'bagging_fraction': 0.8612478367292172, 'bagging_freq': 6, 'min_child_samples': 52}
Best accuracy: 0.6695738520047445
[LightGBM] [Warning] Unknown parameter: silent
[LightGBM] [Warning] feature_fraction is set=0.9693790705619205, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9693790705619205
[LightGBM] [Warning] lambda_l2 is set=0.002326714730678544, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.002326714730678544
[LightGBM] [Warning] lambda_l1 is set=0.003109085161653011, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.003109085161653011
[LightGBM] [Warning] bagging_fraction is set=0.8612478367292172, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8612478367292172
[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6


[I 2024-03-22 07:23:18,652] A new study created in memory with name: no-name-5ffede26-5d14-4f55-9cd0-03041e8dc093


Tuning LGBM for target: Other_Faults


[I 2024-03-22 07:23:21,124] Trial 0 finished with value: 0.6104908033763726 and parameters: {'lambda_l1': 1.0812873098067542e-05, 'lambda_l2': 9.68416126930446e-08, 'num_leaves': 122, 'feature_fraction': 0.6777555504103315, 'bagging_fraction': 0.9599487005920896, 'bagging_freq': 1, 'min_child_samples': 23}. Best is trial 0 with value: 0.6104908033763726.
[I 2024-03-22 07:23:22,763] Trial 1 finished with value: 0.6193795634088445 and parameters: {'lambda_l1': 0.0005495879692070994, 'lambda_l2': 0.13774955623611682, 'num_leaves': 86, 'feature_fraction': 0.4764407278478008, 'bagging_fraction': 0.7717946661618985, 'bagging_freq': 3, 'min_child_samples': 91}. Best is trial 1 with value: 0.6193795634088445.
[I 2024-03-22 07:23:24,306] Trial 2 finished with value: 0.6160388170416349 and parameters: {'lambda_l1': 8.544616630173735e-08, 'lambda_l2': 5.899086045403207e-05, 'num_leaves': 45, 'feature_fraction': 0.7178449759236574, 'bagging_fraction': 0.9938268160391807, 'bagging_freq': 7, 'min_ch

Best hyperparameters: {'lambda_l1': 0.06831619215094344, 'lambda_l2': 0.0011288861142236535, 'num_leaves': 63, 'feature_fraction': 0.4910560075676224, 'bagging_fraction': 0.7634053836888786, 'bagging_freq': 5, 'min_child_samples': 39}
Best accuracy: 0.6227853522088905
[LightGBM] [Warning] Unknown parameter: silent
[LightGBM] [Warning] feature_fraction is set=0.4910560075676224, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4910560075676224
[LightGBM] [Warning] lambda_l2 is set=0.0011288861142236535, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.0011288861142236535
[LightGBM] [Warning] lambda_l1 is set=0.06831619215094344, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.06831619215094344
[LightGBM] [Warning] bagging_fraction is set=0.7634053836888786, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7634053836888786
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
